# **Feature Selection**


## Selection based on feature variance
elimina todas las características que tienen una varianza pequeña; normalmente, menor que la establecida.

In [18]:
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
import numpy as np


# construir un conjunto artificial de datos
X, y = make_classification(n_samples=10, n_features=5,
                           n_informative=3, n_redundant=0, random_state=101)


print(np.var(X, axis=0))


[1.36033407 1.36888812 0.94894438 0.6457313  1.73496904]


La menor varianza está asociada a la  característica 4; por lo tanto, si
queremos seleccionar las cuatro mejores características, deberíamos fijar
el umbral de varianza mínima en 0.9  Hagámoslo y veamos qué ocurre con
la primera observación del conjunto de datos:

In [19]:
from sklearn.feature_selection import VarianceThreshold
X_selected = VarianceThreshold(threshold=0.9).fit_transform(X)
print ("Before:", X[0, :])
print ("After: ", X_selected[0, :]) 

Before: [-2.02411927 -1.66432922 -0.82764946  0.05353877  0.34858907]
After:  [-2.02411927 -1.66432922 -0.82764946  0.34858907]


## Selección univariante
se pretende seleccionar lasvariables individuales que más se asocian con su variable objetivo según
una prueba estadística.


In [27]:
X, y = make_classification(n_samples=800, n_features=100, n_informative=25,
                           n_redundant=0, random_state=101)

array([[-0.84933815,  0.48925586,  0.72980944],
       [ 1.36658414,  2.04664397,  0.17731243],
       [ 1.64266792, -0.43187942, -0.90353033],
       [ 1.23495751, -2.76593955,  0.06069002]])

In [44]:
from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection import chi2, f_classif
from sklearn.preprocessing import Binarizer, scale
import numpy as np

# En chi-cuadrado la entrada X debe ser no negativa(debe contener booleanos ofrecuencias), de
#  ahí la elección de binarizar luego de normalizar si la variable está por encima de la media.
Xbin = Binarizer().fit_transform(scale(X))

Selector_chi2 = SelectPercentile(chi2, percentile=25).fit(Xbin, y)
Selector_f_classif = SelectPercentile(f_classif, percentile=25).fit(X, y)

chi_scores = Selector_chi2.get_support()
f_classif_scores = Selector_f_classif.get_support()

# seleccionar las caracteristicas que estan en los dos test
selected = chi_scores & f_classif_scores  # use the bitwise and operator

selected.nonzero()


(array([ 1,  6,  7, 12, 30, 34, 35, 38, 45, 54, 56, 59, 61, 66, 72, 76, 80,
        81, 84, 89, 99], dtype=int64),)

## Selection based on recursive elimination
En cada iteracion va podando variables que mantengan estable o aumenten la puntuacion que se este utilizando.cuando se observa una gran discrepancia entre los resultados de entrenamiento (basados en la validación cruzada, no en la puntuación
dentro de la muestra) y los resultados fuera de la muestra, **la selección recursiva** puede ayudar a conseguir un mejor rendimiento de los algoritmos de aprendizaje, señalando algunas de las variables más importantes.

In [55]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=100, n_features=100,
                           n_informative=5, n_redundant=2, random_state=101)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=101)


In [57]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state=101)
classifier.fit(X_train, y_train)
print ('In-sample accuracy: %0.3f' % classifier.score(X_train, y_train))
print ('Out-of-sample accuracy: %0.3f' % classifier.score(X_test, y_test))

In-sample accuracy: 1.000
Out-of-sample accuracy: 0.767


In [56]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFECV

classifier = LogisticRegression(random_state=101)
classifier.fit(X_train, y_train)


selector = RFECV(estimator=classifier, step=1, cv=10, scoring='accuracy')
selector.fit(X_train, y_train)
print('Optimal number of features : %d' % selector.n_features_)

# trasformar los datos ya con las cararacteristica sseleccionadas
X_train_s = selector.transform(X_train)
X_test_s = selector.transform(X_test)
classifier.fit(X_train_s, y_train)
print('Out-of-sample accuracy: %0.3f' %
      classifier.score(X_test_s, y_test))


Optimal number of features : 33
Out-of-sample accuracy: 0.700
